

# Saving/Resuming Study with RDB Backend

An RDB backend enables persistent experiments (i.e., to save and resume a study) as well as access to history of studies.
In addition, we can run multi-node optimization tasks with this feature, which is described in `distributed`.

In this section, let's try simple examples running on a local environment with SQLite DB.

<div class="alert alert-info"><h4>Note</h4><p>You can also utilize other RDB backends, e.g., PostgreSQL or MySQL, by setting the storage argument to the DB's URL.
    Please refer to [SQLAlchemy's document](https://docs.sqlalchemy.org/en/latest/core/engines.html#database-urls) for how to set up the URL.</p></div>


## New Study

We can create a persistent study by calling :func:`~optuna.study.create_study` function as follows.
An SQLite file ``example.db`` is automatically initialized with a new study record.


In [1]:
import logging
import sys

import optuna

# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "example-study"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)
study = optuna.create_study(study_name=study_name, storage=storage_name)

[I 2023-08-08 15:11:02,874] A new study created in RDB with name: example-study


A new study created in RDB with name: example-study


To run a study, call :func:`~optuna.study.Study.optimize` method passing an objective function.



In [2]:
def objective(trial):
    x = trial.suggest_float("x", -10, 10)
    return (x - 2) ** 2


study.optimize(objective, n_trials=3)

[I 2023-08-08 15:11:04,102] Trial 0 finished with value: 55.041430977846915 and parameters: {'x': -5.418991237213245}. Best is trial 0 with value: 55.041430977846915.


Trial 0 finished with value: 55.041430977846915 and parameters: {'x': -5.418991237213245}. Best is trial 0 with value: 55.041430977846915.


[I 2023-08-08 15:11:04,185] Trial 1 finished with value: 1.2260697767496562 and parameters: {'x': 0.8927196485308446}. Best is trial 1 with value: 1.2260697767496562.


Trial 1 finished with value: 1.2260697767496562 and parameters: {'x': 0.8927196485308446}. Best is trial 1 with value: 1.2260697767496562.


[I 2023-08-08 15:11:04,278] Trial 2 finished with value: 23.543922041302594 and parameters: {'x': 6.8522079552820685}. Best is trial 1 with value: 1.2260697767496562.


Trial 2 finished with value: 23.543922041302594 and parameters: {'x': 6.8522079552820685}. Best is trial 1 with value: 1.2260697767496562.


## Resume Study

To resume a study, instantiate a :class:`~optuna.study.Study` object
passing the study name ``example-study`` and the DB URL ``sqlite:///example-study.db``.



In [3]:
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True)
study.optimize(objective, n_trials=3)

[I 2023-08-08 15:11:10,278] Using an existing study with name 'example-study' instead of creating a new one.


Using an existing study with name 'example-study' instead of creating a new one.


[I 2023-08-08 15:11:10,497] Trial 3 finished with value: 2.7101006945602473 and parameters: {'x': 0.35376165317404684}. Best is trial 1 with value: 1.2260697767496562.


Trial 3 finished with value: 2.7101006945602473 and parameters: {'x': 0.35376165317404684}. Best is trial 1 with value: 1.2260697767496562.


[I 2023-08-08 15:11:10,617] Trial 4 finished with value: 6.2528332784699785 and parameters: {'x': -0.5005665914888127}. Best is trial 1 with value: 1.2260697767496562.


Trial 4 finished with value: 6.2528332784699785 and parameters: {'x': -0.5005665914888127}. Best is trial 1 with value: 1.2260697767496562.


[I 2023-08-08 15:11:10,744] Trial 5 finished with value: 1.8854102177498848 and parameters: {'x': 0.6268975938591161}. Best is trial 1 with value: 1.2260697767496562.


Trial 5 finished with value: 1.8854102177498848 and parameters: {'x': 0.6268975938591161}. Best is trial 1 with value: 1.2260697767496562.


Note that the storage doesn't store the state of the instance of :mod:`~optuna.samplers`.
When we resume a study with a sampler whose ``seed`` argument is specified for
reproducibility, you need to restore the sampler with using ``pickle`` as follows::

    import pickle

    # Save the sampler with pickle to be loaded later.
    with open("sampler.pkl", "wb") as fout:
        pickle.dump(study.sampler, fout)

    restored_sampler = pickle.load(open("sampler.pkl", "rb"))
    study = optuna.create_study(
        study_name=study_name, storage=storage_name, load_if_exists=True, sampler=restored_sampler
    )
    study.optimize(objective, n_trials=3)




## Experimental History

We can access histories of studies and trials via the :class:`~optuna.study.Study` class.
For example, we can get all trials of ``example-study`` as:



In [4]:
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=True)
df = study.trials_dataframe(attrs=("number", "value", "params", "state"))

[I 2023-08-08 15:11:15,076] Using an existing study with name 'example-study' instead of creating a new one.


Using an existing study with name 'example-study' instead of creating a new one.


The method :func:`~optuna.study.Study.trials_dataframe` returns a pandas dataframe like:



In [5]:
print(df)

   number      value  params_x     state
0       0  55.041431 -5.418991  COMPLETE
1       1   1.226070  0.892720  COMPLETE
2       2  23.543922  6.852208  COMPLETE
3       3   2.710101  0.353762  COMPLETE
4       4   6.252833 -0.500567  COMPLETE
5       5   1.885410  0.626898  COMPLETE


A :class:`~optuna.study.Study` object also provides properties
such as :attr:`~optuna.study.Study.trials`, :attr:`~optuna.study.Study.best_value`,
:attr:`~optuna.study.Study.best_params` (see also `first`).



In [ ]:
print("Best params: ", study.best_params)
print("Best value: ", study.best_value)
print("Best Trial: ", study.best_trial)
print("Trials: ", study.trials)